# Example Q7: Single Shot Fidelity
This example notebook shows how to run single shot fidelity experiments

© Raytheon BBN Technologies 2021

In [1]:
from QGL import *
from auspex.qubit import *

auspex-WARNING: 2021-04-14 22:42:31,718 ----> You may not have the libusb backend: please install it!
auspex-WARNING: 2021-04-14 22:42:32,035 ----> Could not load channelizer library; falling back to python methods.


In [2]:
cl = ChannelLibrary(":memory:")

q1 = cl.new_qubit("q1")
aps2_1 = cl.new_APS2("BBNAPSa", address="192.168.2.4", trigger_interval=200e-6) 
aps2_2 = cl.new_APS2("BBNAPSb", address="192.168.2.2")
dig_1  = cl.new_X6("Dig_1", address="1", sampling_rate=500e6, record_length=1024)
h1 = cl.new_source("Holz_1", "HolzworthHS9000", "HS9004A-009-1", reference='10MHz', power=-30)
h2 = cl.new_source("Holz_2", "HolzworthHS9000", "HS9004A-009-2", reference='10MHz', power=-30)

cl.set_measure(q1, aps2_1, dig_1.ch(1), trig_channel=aps2_1.ch("m2"), gate=False, generator=h1)
cl.set_control(q1, aps2_2, generator=h2)
cl.set_master(aps2_1, aps2_1.ch("m1"))
cl["q1"].measure_chan.frequency = 0e6
cl["q1"].measure_chan.autodyne_freq = 10e6

In [3]:
# setup the pipeline
def create_fidelity_pipeline():
    pl = PipelineManager()
    pl.create_default_pipeline(qubits=(cl['q1'],))
    for ql in ['q1']:
        qb = cl[ql]
        pl[ql].clear_pipeline()
        pl[ql].stream_type = "raw"
        pl[ql].create_default_pipeline(buffers=False) 
        pl[ql].if_freq = qb.measure_chan.autodyne_freq
        pl[ql]["Demodulate"].frequency = qb.measure_chan.autodyne_freq
        pl[ql].add(FidelityKernel(save_kernel=True, logistic_regression=False, set_threshold=True, label=f"Q{ql[-1]}_SSF"))
        pl[ql]["Demodulate"]["Integrate"].simple_kernel = True
        pl[ql]["Demodulate"]["Integrate"].box_car_start = 0.4e-7
        pl[ql]["Demodulate"]["Integrate"].box_car_stop = 1.2e-6
        #pl[ql]["Demodulate"]["Integrate"].add(Write(label="RR-Writer", groupname=ql+"-int"))
        pl[ql]["Demodulate"]["Integrate"]["Average"].add(Display(label=ql+" - Final Average", plot_dims=0))
        pl[ql]["Demodulate"]["Integrate"]["Average"].add(Display(label=ql+" - Partial Average", plot_dims=0), connector_out="partial_average")
    return pl

In [4]:
pl = create_fidelity_pipeline()
pl.show_pipeline()

auspex-INFO: 2021-04-14 22:42:42,780 ----> Could not find an existing pipeline. Creating a blank pipeline.


Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', width='960…

In [5]:
exp = SingleShotFidelityExperiment(q1, averages=20000, exp_name="Q1_SSF")
# f = 6.800e9 - cl[qubit].measure_chan.autodyne_freq
# f = cl['Autodyne_Mq1'].frequency = 6.6895e9 # 6688895477 # 6689263627 # 6689631777 # 6.6893e9
# span = 1e6
# exp = QubitExperiment(PulsedSpec(cl[qubit], specOn=False), averages=256)
# exp.add_instrument_sweep('Autodyne_M' + 'q1', 'frequency', np.arange(f-span, f+span, 0.1e6))
# exp.add_instrument_sweep('Autodyne_M' + 'q1', '')
# exp.run_sweeps()

Compiled 2 sequences.
auspex-INFO: 2021-04-14 22:42:46,209 ----> Creating a new sample in the calibration database.
